In [1]:
import numpy as np

class MF():

    def __init__(self, R, K, alpha, beta, iterations):

        self.R = R #user-item матрица
        self.num_users, self.num_items = R.shape 
        self.K = K #латентные факторы
        self.alpha = alpha #learning rate
        self.beta = beta #регуляризация
        self.iterations = iterations

    def train(self):
        #Инициализируем латентные параметры
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # Инициализируем bias
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])

        #Создаем лист для тренировочных записей
        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]

        #Выполняем SGD для заданного количества итераций
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            if (i+1) % 10 == 0:
                print("Iteration: %d ; error = %.4f" % (i+1, mse))

        return training_process

    def mse(self):
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    def sgd(self):
        for i, j, r in self.samples:
            #Prediction, error
            prediction = self.get_rating(i, j)
            e = (r - prediction)

            #Bias
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

            #Обновляем матрицы
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    def get_rating(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    def full_matrix(self):
        return self.b + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)

In [2]:
R = np.array([
    [5, 3, 0, 1],
    [4, 0, 0, 1],
    [1, 1, 0, 5],
    [1, 0, 0, 4],
    [0, 1, 5, 4],
])

mf = MF(R, K=2, alpha=0.1, beta=0.01, iterations=30)
training_process = mf.train()

Iteration: 10 ; error = 0.3543
Iteration: 20 ; error = 0.0711
Iteration: 30 ; error = 0.0413


In [3]:
mf.full_matrix()

array([[4.98431654, 3.00068683, 3.74109598, 1.00934121],
       [3.99662655, 2.27345855, 3.16663176, 1.01114107],
       [1.01551181, 1.00025236, 3.98666316, 4.98666254],
       [1.00472823, 0.77756253, 3.22096071, 3.99119607],
       [1.60260131, 1.02406783, 4.989066  , 3.99548892]])

In [4]:
np.round(mf.full_matrix())

array([[5., 3., 4., 1.],
       [4., 2., 3., 1.],
       [1., 1., 4., 5.],
       [1., 1., 3., 4.],
       [2., 1., 5., 4.]])